In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
from imutils import paths

In [ ]:
IMG_SIZE = 256
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 8

In [ ]:
# For keypoints
def unnormalize_and_scale_labels(keypoints, max_val):
    keypoints = tf.math.scalar_mul(max_val, tf.math.add(keypoints, 0.5))
    return keypoints

In [ ]:
def read_tfrecord(example):
    # feature map for extraction of data from TFRecord
    features = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'image_mask': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([42], tf.float32),
    }
    image_features = tf.io.parse_single_example(example, features)

    # For images
    image = tf.io.decode_raw(image_features['image_raw'], out_type = tf.uint8)
    image = tf.reshape(image, [256, 256, 3])

    # For keypoints
    labels = tf.reshape(image_features['label'], [21, 2])
    labels = unnormalize_and_scale_labels(labels, IMG_SIZE)
    labels = tf.dtypes.cast(labels, tf.float32)
    return image, labels


In [ ]:
def load_dataset(filenames):
    """
    Load each TFRecord
    """
    ignore_order = tf.data.Options()
    ignore_order.deterministic = False

    files = tf.data.Dataset.list_files(filenames)
    dataset = files.with_options(ignore_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=512, num_parallel_calls=AUTO)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
    return dataset

In [ ]:
tran_dir = "/content/drive/MyDrive/archive/train"
train_tfrecord_names = list(paths.list_files("/content/drive/MyDrive/archive/train"))

In [ ]:
dataset = load_dataset(filenames)